<a href="https://colab.research.google.com/github/Aggraj/Deep-Learning-CS-6910/blob/main/Copy_of_FFNNv7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np

class Feedforwardneuralnetwork:

    def __init__(self,n_inputs,n_hidden,n_outputs,activation):

        self.n_inputs   = n_inputs
        self.n_outputs  = n_outputs
        self.n_hidden   = n_hidden
        self.activation = activation
        self.weights    = []
        self.biases     = []


        layers = [self.n_inputs] + self.n_hidden + [self.n_outputs]
        for i in range(len(n_hidden)+1):
         #  self.weights.append(np.random.randn(layers[i+1],layers[i]))
            self.weights = [np.random.randn(y, x)* 2/np.sqrt(x+y) for x,y in zip(layers[:-1], layers[1:])]
            self.biases.append(np.random.randn(layers[i+1],1))



    def sigmoid(self,x):
        return 1 / ( 1 + np.exp(-x))

    def tanh(self,x):
	      return np.tanh(x)
       
    def d_tanh(self,x):
        return (1 - (np.tanh(x))**2)
    
    def relu(self, x):
        return np.maximum(0, x, x)
        
    def d_relu(x):
        return np.greater(x,0).astype(int)   

    def softmax(self,x):
        soft = np.zeros(x.shape)
        for i in range(0, x.shape[1]):
            numr = np.exp(x[:, i])
            soft[:, i] = numr/np.sum(numr)
        return soft

    def forward_propagation(self,input):

        self.intermidiate_inputs = []
        self.post_outputs  = []
        W      = self.weights
        b      = self.biases

        k=0
        self.intermidiate_inputs.append(np.matmul(W[k],input)+b[k])
        if self.activation == 'sigmoid':
          self.post_outputs.append(self.sigmoid(self.intermidiate_inputs[k]))
        elif self.activation == 'tanh':
          self.post_outputs.append(self.tanh(self.intermidiate_inputs[k]))
        elif self.activation == 'relu': 
          self.post_outputs.append(self.Relu(self.intermidiate_inputs[k])) 

        for k in range(1,len(self.n_hidden)):
            self.intermidiate_inputs.append(np.matmul(W[k],self.post_outputs[k-1])+b[k])
            if self.activation == 'sigmoid':
              self.post_outputs.append(self.sigmoid(self.intermidiate_inputs[k]))
            elif self.activation == 'tanh':
              self.post_outputs.append(self.tanh(self.intermidiate_inputs[k]))
            elif self.activation == 'relu': 
              self.post_outputs.append(self.Relu(self.intermidiate_inputs[k])) 


        k=len(self.n_hidden)
        self.intermidiate_inputs.append(np.matmul(W[k],self.post_outputs[k-1])+b[k])
        self.post_outputs.append(self.softmax(self.intermidiate_inputs[k]))

        return self.post_outputs[-1]

    def back_propagation(self,train_images,train_labels):

        g_weights = [0]*(len(self.weights))
        g_biases  = [0]*(len(self.biases))
        g_a       = [0]*(len(self.n_hidden)+1)
        g_h       = [0]*(len(self.n_hidden)+1)
        n_samples = train_images.shape[0]  # Change depending on the dimensions of data


        for k in reversed(range(len(self.n_hidden)+1)):
            if k == len(self.n_hidden):
                g_a[k] = self.post_outputs[k] - train_labels  # keep or remove T depending on the dimensions of data
                #g_weights[k] = (1/n_samples)*np.matmul(g_a[k],self.post_outputs[k-1].T)
                #g_biases[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
            else:
                g_h[k] = (1/n_samples)*np.matmul(self.weights[k+1].T,g_a[k+1])
                if self.activation == 'sigmoid':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],np.multiply(self.sigmoid(self.intermidiate_inputs[k]),(1-self.sigmoid(self.intermidiate_inputs[k]))))
                elif self.activation == 'tanh':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],self.d_tanh(self.intermidiate_inputs[k]))
                elif self.activation == 'relu':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],self.d_relu(self.intermidiate_inputs[k]))

            if k == 0:
                g_weights[k] = (1/n_samples)*np.matmul(g_a[k],train_images.T) 
            else:
                g_weights[k] = (1/n_samples)*np.matmul(g_a[k],self.post_outputs[k-1].T)

            g_biases[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
        return g_weights,g_biases



    def train_model(self,train_images,train_labels,train_val_images,train_val_labels,epochs,learning_rate,opt='gd',batch_size = 100): 
         
      pre_delta_w = np.multiply(self.weights,0)
      pre_delta_b = np.multiply(self.biases,0)
      delta_w = np.multiply(self.weights,0)
      delta_b = np.multiply(self.biases,0)
      vw = 0.0
      vb = 0.0
      eps = 1e-8
      lr_w = 0.0
      lr_b = 0.0
      gamma = 0.9
      beta = 0.999
      beta1 = 0.9
      beta2 = 0.999
      m_t, v_t, m_hat_w, v_hat_w, m_b,v_b,m_hat_b,v_hat_b = 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0 

      for i in range(epochs+1):
            
            for bb in range(0, train_images.shape[1], batch_size):

              train_b_imag = train_images[:,bb:bb+batch_size]
              train_l_imag = train_labels[:,bb:bb+batch_size]
              output =  self.forward_propagation(train_b_imag)
              g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)
              if opt == 'gd':
                 delta_w = np.multiply(learning_rate,g_weights)
                 delta_b = np.multiply(learning_rate,g_biases)
          
              if opt == 'mgd':
                 delta_w = np.multiply(gamma,pre_delta_w) + np.multiply(learning_rate,g_weights)
                 delta_b = np.multiply(gamma,pre_delta_b) + np.multiply(learning_rate,g_biases)
                 pre_delta_w = delta_w
                 pre_delta_b = delta_b

              if opt == 'ngd':
                 self.weights = self.weights - np.multiply(gamma,pre_delta_w)
                 self.biases  = self.biases - np.multiply(gamma,pre_delta_b)
                #output =  self.forward_propagation(train_b_imag)
                 g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)

                 delta_w = np.multiply(gamma,pre_delta_w) + np.multiply(learning_rate,g_weights)
                 delta_b = np.multiply(gamma,pre_delta_b) + np.multiply(learning_rate,g_biases)
                
                 pre_delta_w = delta_w
                 pre_delta_b = delta_b
                
              if opt == 'rmsprop': 
                 
                 vw = np.multiply(vw,beta) + np.multiply(1-beta,np.power(g_weights,2))
                 vb = np.multiply(vb,beta) + np.multiply(1-beta,np.power(g_biases,2))
                 lr_w = learning_rate/np.power(vw+eps,1/2)
                 lr_b = learning_rate/np.power(vb+eps,1/2)
          
                 delta_w = np.multiply(g_weights,lr_w)
                 delta_b = np.multiply(g_biases,lr_b)
              
              if opt == 'adam':
                 m_t = np.multiply(beta1,m_t) + np.multiply((1-beta1),self.weights)
                 v_t = np.multiply(beta2,v_t) + np.multiply((1-beta2),np.power(self.weights,2))
                 m_b = np.multiply(beta1,m_b) + np.multiply((1-beta1),self.biases)
                 v_b = np.multiply(beta2,v_b) + np.multiply((1-beta2),np.power(self.biases,2))
                
                 m_hat_w = m_t/(1 - np.power(beta1,i+1))
                 m_hat_b = m_b/(1 - np.power(beta1,i+1))
                
                 v_hat_w = v_t/(1 - np.power(beta2,i+1))
                 v_hat_b = v_b/(1 - np.power(beta2,i+1))
                 delta_w = (learning_rate / np.power(v_hat_w + eps, 1/2)) * m_hat_w
                 delta_b = (learning_rate / np.power(v_hat_b + eps, 1/2)) * m_hat_b
              if opt == 'nadam':

                 g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)
                 m_t = np.multiply(beta1, m_t) + np.multiply((1 - beta1),g_weights)
                 v_t =  np.multiply(beta2,v_t) +  np.multiply((1 - beta2),np.power(g_weights, 2))

                 m_b =  np.multiply(beta1,m_b) + np.multiply((1 - beta1),g_biases)
                 v_b = np.multiply(beta2,v_b) + np.multiply((1 - beta2),np.power(g_biases, 2))
                
                 m_hat_t = m_t / (1 - np.power(beta1, i+1)) + np.multiply((1 - beta1),g_weights / (1 - np.power(beta1, i+1)))
                 v_hat_t = v_t / (1 - np.power(beta2, i+1))

                 m_hat_b = m_b / (1 - np.power(beta1, i+1)) + np.multiply((1 - beta1),g_biases / (1 - np.power(beta1, i+1)))
                 v_hat_b = v_b / (1 - np.power(beta2, i+1))

                 delta_w =  np.multiply(learning_rate,m_hat_w / (np.power(v_hat_w + eps,1/2)))
                 delta_b =  np.multiply(learning_rate,m_hat_b / (np.power(v_hat_b + eps,1/2)))

             
              self.weights = self.weights - delta_w
              self.biases  = self.biases  - delta_b
                
              train_loss = -np.sum(np.multiply(train_l_imag,np.log(output)))/train_l_imag.shape[1]    
            #print('training_loss for epoch {} = {}'.format(i,train_loss))
            
            output = self.forward_propagation(train_images)
            out_class=(np.argmax(output,axis=0))
            target_class=(np.argmax(train_label,axis=1))
            acc1 = 100*np.sum(out_class==target_class)/output.shape[1]
            
            Validate = self.forward_propagation(train_val_images)
            out_class=(np.argmax(Validate,axis=0))
            target_class_validate=(np.argmax(train_val_labels,axis=1))
            acc2 = 100*np.sum(out_class==target_class_validate)/Validate.shape[1]
            print('Epoch {}: training_accuracy = {:.2f}, Validation accuracy = {:.2f}'.format(i,acc1,acc2))


      return acc2

In [16]:
import tensorflow as tf
from sklearn.metrics import log_loss
from keras.datasets import fashion_mnist
output_classes = 10
activation = 'tanh'
Mode  = Feedforwardneuralnetwork(28*28,[150,200],output_classes,'tanh')
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
n_samples = train_images.shape[0]
train_images = train_images.reshape(n_samples,-1)
train_imag = train_images[:54000,:]
train_val_images = train_images[54000:,:]
labels = np.zeros((train_labels.shape[0],output_classes))
for i in range(train_labels.shape[0]):
  e = [0.0]*output_classes
  e[train_labels[i]] = 1.0
  labels[i] = e
train_label = labels[:54000,:]
train_val_labels = labels[54000:,:]
mean = train_imag.mean(axis=0)
std  = train_imag.std(axis = 0)
train_imag = (train_imag - mean)/std
train_val_images = (train_val_images - mean)/std
#epochs = 10
#learning_rate = 0.01
#(tr_loss) = Mode.train_model(train_imag.T,train_label.T,train_val_images.T,train_val_labels,epochs,learning_rate,'rmsprop')

In [17]:
%pip install wandb -q
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [18]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values': [5,10,15]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4]
        },
        'opt': {
            'values': ['adam','nadam','gd', 'rmsprop','mgd','ngd']
        },
        'activation': {
            'values': ['relu', 'tanh', 'sigmoid']
        },
        'n_hidden': {
            'values': [ [32,64],[32,64,128],[100,150],[32,64,128,256]]
        },
        'batch_size':{
            'values':[16,32,64,128]
        }
    }
}


In [19]:
sweep_id = wandb.sweep(sweep_config, entity="chaxin", project="Assignment 1")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: ucs3kdeq
Sweep URL: https://wandb.ai/chaxin/Assignment%201/sweeps/ucs3kdeq


In [21]:
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 2,
        'learning_rate': 1e-3,
        'hidden':[100,200],
        'learning_rate':1e-2,
        'opt':'ngd',
        'activation':'sigmoid',
        'n_inputs': 28*28,
        'n_outputs': 10,
        'batch_size':100
    }

    # Initialize a new wandb run
    wandb.init(project='Assignment 1', entity='chaxin',config=config_defaults)
    
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    learning_rate = config.learning_rate
    epochs = config.epochs
    n_hidden = config.hidden
    activation = config.activation
    opt = config.opt
    n_inputs = config.n_inputs
    n_outputs = config.n_outputs
    batch_size = config.batch_size
    # Model training here
    sweep_network    = Feedforwardneuralnetwork(n_inputs, n_hidden, n_outputs,activation)
    accuracy = sweep_network.train_model(train_imag.T,train_label.T,train_val_images.T,train_val_labels,epochs,learning_rate,opt,batch_size)

#train_network(network, dataset, config.learning_rate, config.epochs, n_outputs)




# 3. Log metrics over time to visualize performance
    wandb.log({"accuracy": accuracy})

In [ ]:
wandb.agent(sweep_id, train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: i2bnhtd1 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0001
wandb: 	n_hidden: [32, 64, 128]
wandb: 	opt: ngd
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:112: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:113: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:114: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/lo

Epoch 0: training_accuracy = 9.93, Validation accuracy = 9.95
Epoch 1: training_accuracy = 7.23, Validation accuracy = 7.88
Epoch 2: training_accuracy = 5.43, Validation accuracy = 5.87
